In [1]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=ca0226ff64ee285d6801156941441df993473d1f32707a707fddf7dc7faf04dc
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [2]:
!pip install nltk

In [3]:
import nltk
import random
import string
import re, string, unicodedata
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import wikipedia as wk
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
data=open('/content/HR.txt.txt','r',errors='ignore')
raw=data.read()
data1=open('/content/CV.txt','r',errors='ignore')
raw1=data1.read()
raw=raw+raw1
raw=raw.lower()

In [0]:
sent_tokens=nltk.sent_tokenize(raw)

In [0]:
def Normalize(text):
  remove_punct_dict= dict((ord(punct),None) for punct in string.punctuation)
  ##### WORD TOKENIZATION
  word_token = nltk.word_tokenize(text.lower().translate(remove_punct_dict))

  ##### REMOVE ASCII
  new_words = []
  for word in word_token:
    new_word = unicodedata.normalize('NFKD',word).encode('ascii','ignore').decode('utf-8','ignore')
    new_words.append(new_word)

  ##### REMOVE TAGS
  rmv=[]
  for w in new_words:
    text=re.sub("&lt;/?.*?&gt;","&lt;&gt;",w)
    rmv.append(text)

  ##### POS TAGGING AND LEMMATIZATION
  tag_map = defaultdict(lambda : wn.NOUN)
  tag_map['J'] = wn.ADJ
  tag_map['V'] = wn.VERB
  tag_map['R'] = wn.ADV
  lmtzr = WordNetLemmatizer()
  lemma_list = []
  rmv = [i for i in rmv if i]
  for token, tag in nltk.pos_tag(rmv):
    lemma = lmtzr.lemmatize(token, tag_map[tag[0]])
    lemma_list.append(lemma)
  return lemma_list

In [0]:
welcome_input = ("hello","hi","greetings","sup","what's up","hey",)
welcome_response = ["hi","hey","*nods*","hi there","hello","I am glad! You are talking to me"]
def welcome(user_response):
  for word in user_response.split():
    if word.lower() in welcome_input:
      return random.choice(welcome_response)

In [0]:
def generateResponse(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=Normalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    #vals = cosine_similarity(tfidf[-1], tfidf)
    vals = linear_kernel(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0) or "tell me about" in user_response:
        print("Checking Wikipedia")
        if user_response:
            robo_response = wikipedia_data(user_response)
            return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response
#wikipedia search
def wikipedia_data(input):
    reg_ex = re.search('tell me about (.*)' or 'what is (.*)', input)
    try:
        if reg_ex:
            topic = reg_ex.group(1)
            wiki = wk.summary(topic, sentences = 3)
            return wiki
    except Exception as e:
            print("No content has been found")

In [15]:
flag=True
print("""HELLO! My Name is Jarvis and I'm a Chatbot. If you want to exit, type "Bye!" """)
while (flag==True):
  user_response = input()
  user_response = user_response.lower()
  if(user_response not in ['bye','shutdown','exit','quit']):
    if(user_response=='thanks' or user_response=='thank you'):
      flag=False
      print("JARVIS: You are welcome.")
    else:
        if(welcome(user_response)!=None):
          print("JARVIS: "+welcome(user_response))
        else:
            print("JARVIS: ",end="")
            print(generateResponse(user_response))
            sent_tokens.remove(user_response)

  else:
    flag=False
    print("JARVIS: Bye!!!")

HELLO! My Name is Jarvis and I'm a Chatbot. If you want to exit, type "Bye!" 
hey
JARVIS: hi
tell me about coronavirus
JARVIS: Checking Wikipedia
Coronaviruses are a group of related RNA viruses that cause diseases in mammals and birds. In humans, these viruses cause respiratory tract infections that can range from mild to lethal. Mild illnesses include some cases of the common cold (which is caused also by certain other viruses, predominantly rhinoviruses), while more lethal varieties can cause SARS, MERS, and COVID-19.
what is maternity leave
JARVIS: maternity leave
according to the section 5 of the maternity benefit act 1961, it is woman employee right to claim for maternity leave for 26 weeks for 2. such woman employee can apply maternity leave just eight weeks before the date of are expected delivery.
what are the primary responsibilities of human resource manager
JARVIS: primary responsibilities of the human resource manager:
to develop a thorough knowledge of corporate culture, 